# Multithreading

In the preceding examples, csound was executed within a performance loop and any python code needed to be run within that loop. To use Csound in a more flexible way it is advisable to run the performance loop in a dedicated thread

Cound has a helper class called CsoundPerformanceThread, which creates a native thread and runs the performance loop of an existing csound instance on the background. The main Python thread is thus not blocked, allowing the user to interract with it, while the performance thread runs concurrently, outside of the GIL. The user can send messages to the performance thread to toggle pause, schedule input evets, etc.

## Example


In [1]:
import ctcsound7 as ct
cs = ct.Csound()
cs.setOption('-d -odac -m0')
cs.compileOrc(r'''
sr     = 48000
ksmps  = 64
nchnls = 2
0dbfs  = 1

instr 1
  iamp, ipitch, iattack, idec, ipan passign 4
  aenv = linen:a(1, iattack, p3, idec)
  asig = poscil(iamp, mtof(ipitch)) * aenv
  a1, a2 pan2 asig, ipan
  outs a1, a2
endin
''')


0

This creates a new thread with the existing csound process

In [2]:
thread = cs.performanceThread()
thread.play()

--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
writing 512 sample blks of 64-bit floats to dac
SECTION 1:


Now, we can send messages to the performance thread:

In [3]:
thread.scoreEvent(0, 'i', (1, 0,   1, 0.5, 60, 0.05, 0.3, 0.2))
thread.scoreEvent(0, 'i', (1, 0.5, 1, 0.5, 62, 0.05, 0.3, 0.8))

When we're done, we stop the performance thread:

In [4]:
thread.stop()
thread.join()

		   overall amps:  0.62995  0.62998
	   overall samples out of range:        0        0
0 errors in performance
1066 512 sample blks of 64-bit floats written to dac


1

## Process Callback

A performance thread includes methods to control playback and schedule events. All other tasks must be performed via the csound instance. The problem is that when usign a performance thread, accessing the csound instance directly has an added latency. 

In order to access the csound instance while using the performance thread, it is advisable to set a process callback.


In [1]:
import queue
import time
import ctcsound7 as ct

In [6]:
cs = ct.Csound()

# Change as needed
# cs.setOption('-odac')
cs.setOption('-+rtaudio=jack -odac:Built-in' )

cs.compileOrc(r'''
sr     = 48000
ksmps  = 64
nchnls = 2
0dbfs  = 1

instr 1
  ipitch = p4
  asig = poscil(0.1, mtof(ipitch)) 
  outch 1, asig * linen:a(1, 0.01, p3, 0.2)
endin
''')

thread = cs.performanceThread()
thread.play()


class ProcessHandler:
    def __init__(self, csound):
        self.csound = csound
        self.thread = csound.performanceThread()
        self.q = queue.SimpleQueue()
        self.thread.setProcessCallback(self.callback)
        
    def put(self, job):
        self.q.put(job)
        
    def callback(self, data):
        if self.q.qsize() == 0:
            return
        job = self.q.get_nowait()
        job(self.csound, self.thread)

    def compile(self, code: str):
        self.put(lambda cs, pt: cs.compileOrc(code))

    
proc = ProcessHandler(cs)


rtaudio: JACK module enabled
--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
system sr: 48000.000000
Jack output ports:
 0: dac0 (dac:Built-in Audio Analog Stereo:playback_FL)
 1: dac1 (dac:Built-in Audio Analog Stereo:playback_FR)
connecting channel 0 to Built-in Audio Analog Stereo:playback_FL
connecting channel 1 to Built-in Audio Analog Stereo:playback_FR
writing 512 sample blks of 64-bit floats to dac:Built-in
SECTION 1:


In [7]:
proc.put(lambda cs, pt: cs.compileOrc(r'''
instr 10
  ifreq = p4
  outch 2, vco2:a(1, ifreq) 
endin
'''))

thread.scoreEvent(0, 'i', (1, 0, 1, 72))
# The code is compiled in the next process cycle, so it is possible to schedule
# an event right away
thread.scoreEvent(0, 'i', (10, 0, 3, 600))

  

	   T  1.501 TT  1.501 M:  0.00000  0.00000
new alloc for instr 1:
new alloc for instr 10:


In [2]:
cs = ct.Csound()

# Change as needed
# cs.setOption('-odac')
cs.setOption('-+rtaudio=jack -odac:Built-in' )

cs.compileOrc(r'''
sr     = 48000
ksmps  = 64
nchnls = 2
0dbfs  = 1

instr 1
  ipitch = p4
  asig = poscil(0.1, mtof(ipitch)) 
  outch 1, asig * linen:a(1, 0.01, p3, 0.2)
endin
''')

thread = cs.performanceThread(withProcessQueue=True)
thread.play()

# time.sleep(2)

thread.task(lambda cs, pt: cs.compileOrc(r'''
instr 10
  ifreq = p4
  outch 2, vco2:a(0.1, ifreq) 
endin
'''))

thread.scoreEvent(0, 'i', (1, 0, 1, 72))
thread.scoreEvent(0, 'i', (10, 0, 1, 600))


rtaudio: JACK module enabled
--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
system sr: 48000.000000
Jack output ports:
 0: dac0 (dac:Built-in Audio Analog Stereo:playback_FL)
 1: dac1 (dac:Built-in Audio Analog Stereo:playback_FR)
connecting channel 0 to Built-in Audio Analog Stereo:playback_FL
connecting channel 1 to Built-in Audio Analog Stereo:playback_FR
writing 512 sample blks of 64-bit floats to dac:Built-in
SECTION 1:
	   T  0.001 TT  0.001 M:  0.00000  0.00000
new alloc for instr 1:
new alloc for instr 10:


In [2]:
import ctcsound7 as ct
import time

cs = ct.Csound()
cs.setOption("-+rtaudio=jack -odac -B512 -b256")
cs.compileOrc(r'''
sr = 48000
ksmps = 64
''')

thread = cs.performanceThread(withProcessQueue=True)
thread.play()
bufsize = 1024
time.sleep(0.5)

t0 = time.time()
tabnum = thread.evalCode(f'gi__tabnum ftgen 0, 0, {-bufsize}, -2, 0\nreturn gi__tabnum')
t1 = time.time()
print(tabnum, (t1 - t0) * 1000)

thread.stop()
thread.join()
del cs


rtaudio: JACK module enabled
--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 32768.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
system sr: 48000.000000
Jack output ports:
 0: dac0 (dac:Built-in Audio Analog Stereo:playback_FL)
 1: dac1 (dac:Built-in Audio Analog Stereo:playback_FR)
connecting channel 0 to Built-in Audio Analog Stereo:playback_FL
writing 256 sample blks of 64-bit floats to dac
SECTION 1:


101.0 3.383636474609375


ftable 101:
ftable 101:	1024 points, scalemax 0.000
inactive allocs returned to freespace
		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.560s, CPU: 0.078s
97 256 sample blks of 64-bit floats written to dac


In [3]:
import ctcsound7 as ct
import time

cs = ct.Csound()
cs.setOption("-+rtaudio=jack -odac -b256")
cs.compileOrc(r'''
sr = 48000
ksmps = 64
''')

# csound.compileOrc(...)
thread = cs.performanceThread(withProcessQueue=False)
thread.play()
bufsize = 1024
time.sleep(0.5)
t0 = time.time()

tabnum = cs.evalCode(f'gi__tabnum ftgen 0, 0, {-bufsize}, -2, 0\nreturn gi__tabnum')
t1 = time.time()
print(tabnum, (t1 - t0) * 1000)

thread.stop()
thread.join()
del cs


rtaudio: JACK module enabled
--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 32768.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
system sr: 48000.000000
Jack output ports:
 0: dac0 (dac:Built-in Audio Analog Stereo:playback_FL)
 1: dac1 (dac:Built-in Audio Analog Stereo:playback_FR)
connecting channel 0 to Built-in Audio Analog Stereo:playback_FL
writing 256 sample blks of 64-bit floats to dac
SECTION 1:


101.0 31.600475311279297


ftable 101:
ftable 101:	1024 points, scalemax 0.000
inactive allocs returned to freespace
		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.588s, CPU: 0.050s
105 256 sample blks of 64-bit floats written to dac


In [3]:
thread.compile(f'gi__tabnum ftgen 0, 0, {-bufsize}, -2, 0')

In [4]:
cs.evalCode('return gi__tabnum')

error:  get_arg_type2: Variable 'gi__tabnum' used before defined
Line 0

nan


error:  Variable type for gi__tabnum could not be determined.
Parsing failed due to syntax errors
Stopping on parser failure


In [4]:
cs.evalCode('return i__tabnum')

error:  get_arg_type2: Variable 'i__tabnum' used before defined
Line 0
error:  Variable type for i__tabnum could not be determined.
Parsing failed due to syntax errors
Stopping on parser failure


nan

In [7]:
cs.compileOrc(f'gi__tabnum ftgen 0, 0, {-bufsize}, -2, 0')

ftable 101:
graphics suppressed, ascii substituted
ftable 101:	1024 points, scalemax 0.000


0

In [8]:
cs.evalCode('return gi__tabnum')

101.0